In [2]:
# edit date : 2024-03-05
# version : 1.9.0

from random import randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import *
import os
import time
import datetime as dt
import random
import webbrowser
import subprocess
import win32api


# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC


chrome_path = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s'

############# 자동 예매 원하는 설정으로 변경 ##############

member_number = "2092236381" # 회원번호
password= "thfldjqtdl2@" # 비밀번호
arrival = "수서" # 출발지
departure = "나주" # 도착지
standard_date = "20240503" # 기준날짜 ex) 20221101
standard_time = "16" # 기준 시간 ex) 00 - 22 // 2의 배수로 입력
number_of_trains = 1 # 상단에서부터 조회할 기차수  maximum = 11

#################################################################

def get_driver():
    pathDriver = r"C:\Users\hanta\.wdm\drivers\chromedriver\win64"
    ## r"C:\Users\hanta\.wdm\drivers\chromedriver\win64"
    subfolders = [f.name for f in os.scandir(pathDriver) if f.is_dir()]
    downloaded_latest_driver_version = max(subfolders)
    
    try:
        driver_path = rf"C:\Users\hanta\.wdm\drivers\chromedriver\win64\{downloaded_latest_driver_version}\chromedriver-win32\chromedriver.exe"
        driver = webdriver.Chrome(service=ChromeService(executable_path=driver_path))
        return driver
    except:
        driver_path = ChromeDriverManager().install()
        driver = webdriver.Chrome(service=ChromeService(executable_path=driver_path))
        return driver
def now():
    print(dt.datetime.now())
    
def Repeat_reserved(start_of_trains, number_of_trains):
    reserved = False
    count = 0
    while True: 
        count +=1
        try:
            for i in range(start_of_trains, number_of_trains):
                standard_seat = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(7)").text

                if "예약하기" in standard_seat:
                    print(f"예약 가능 클릭 : {now()}", end='\n')
                    driver.find_element(By.XPATH, f"/html/body/div[1]/div[4]/div/div[3]/div[1]/\
                    form/fieldset/div[6]/table/tbody/tr[{i}]/td[7]/a/span").click()
                    driver.implicitly_wait(3)

                    if driver.find_elements(By.ID, 'isFalseGotoMain'):
                        reserved = True
                        print(f'예약 성공 : {now()}', end='\n')
                        webbrowser.get(chrome_path).open("https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000")
                        win32api.MessageBox(0, "Seat Reserved", "Done", 64)
                        return z
                        # break

                    else:
                        driver.back() #뒤로가기
                        print(f"잔여석 없음. 다시 검색 : {now()}", end='\n')
                        driver.implicitly_wait(5)
 

        except: 
            print('잔여석 조회 불가', end='\n')
            pass

        if not reserved:
            try:
            # 다시 조회하기
                submit = driver.find_element(By.XPATH, "/html/body/div/div[4]/div/div[2]/form/fieldset/div[2]/input")
                driver.execute_script("arguments[0].click();", submit)                              
                print(f"새로고침{count}", end='\r')
                    
            except: 
                print(f"잔여석 없음 #2. 초기화 : {now()}", end='\n')
                driver.back() #뒤로가기
                driver.implicitly_wait(5)

                driver.refresh() #새로고침
                driver.implicitly_wait(5)
                pass

            # 대기
            driver.implicitly_wait(30)
            randomndelay=random.randrange(3000,6001)/1000
            time.sleep(randomndelay)

        else:
            print("오류", end='\n')
            time.sleep(1000)
            break

In [4]:
    driver = get_driver()
    driver.get('https://etk.srail.co.kr/cmc/01/selectLoginForm.do')
    driver.implicitly_wait(15)

    # 회원번호 매핑
    driver.find_element(By.ID, 'srchDvNm01').send_keys(member_number)

    # 비밀번호 매핑
    driver.find_element(By.ID, 'hmpgPwdCphd01').send_keys(password)

    # 확인 버튼 클릭
    driver.find_element(By.XPATH, '/html/body/div/div[4]/div/div[2]/form/\
        fieldset/div[1]/div[1]/div[2]/div/div[2]/input').click()
    driver.implicitly_wait(5)

    driver.get('https://etk.srail.kr/hpg/hra/01/selectScheduleList.do')
    driver.implicitly_wait(5)


    # 출발지 입력
    dep_stn = driver.find_element(By.ID, 'dptRsStnCdNm')
    dep_stn.clear()
    dep_stn.send_keys(arrival)

    # 도착지 입력
    arr_stn = driver.find_element(By.ID, 'arvRsStnCdNm')
    arr_stn.clear()
    arr_stn.send_keys(departure)

    # 날짜 드롭다운 리스트 보이게
    # elm_dptDt = driver.find_element(By.ID, "dptDt")
    # driver.execute_script("arguments[0].setAttribute('style','display: True;)", elm_dptDt)

    Select(driver.find_element(By.ID,"dptDt")).select_by_value(standard_date)

    # 출발 시간
    # eml_dptTm = driver.find_element(By.ID, "dptTm")
    # driver.execute_script("arguments[0].setAttribbute('style','display:True;')", eml_dptTm)

    Select(driver.find_element(By.ID, "dptTm")).select_by_visible_text(standard_time)

    # 조회하기 버튼
    driver.find_element(By.XPATH, "//input[@value='조회하기']").click()


    train_list = driver.find_elements(By.CSS_SELECTOR, "#result-form > fieldset > \
    div.tbl_wrap.th_thead > table > tbody > tr")

In [5]:
standard_date = "20240503" # 기준날짜 ex) 20221101
standard_time = "16" # 기준 시간 ex) 00 - 22 // 2의 배수로 입력
start_of_trains = 1 # 검색 시 보여지는 기차 중 시작할 기차의 서수
number_of_trains = 1 # 상단에서부터 조회할 기차수  maximum = 11
# import win32api

In [ ]:
Repeat_reserved(start_of_trains,number_of_trains)